Librerías

In [ ]:
pip install pandas numpy scikit-learn tensorflow

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import matplotlib.pyplot as plt

Carga y limpieza de los datos

In [ ]:
#Cargamos el dataset en chunks para evitar errores de memoria
chunk_size = 100000  #Cantidad de filas por chunk
chunks = pd.read_csv('accepted_2007_to_2018Q4.csv', chunksize=chunk_size)
#Variables útiles
variables_utiles = [
    'loan_amnt', 'term', 'int_rate', 'installment', 'grade', 'sub_grade',
    'emp_length', 'home_ownership', 'annual_inc', 'verification_status',
    'purpose', 'dti', 'delinq_2yrs', 'fico_range_low', 'fico_range_high',
    'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_record', 'open_acc', 'total_acc', 'revol_bal', 'revol_util',
    'application_type', 'acc_now_delinq', 'pub_rec_bankruptcies', 'mort_acc',
    'num_tl_90g_dpd_24m', 'loan_status', 'total_pymnt_inv', 'chargeoff_within_12_mths'
]
#Combinamos los chunks en un solo DataFrame con solo las variables útiles
df = pd.concat(chunk[variables_utiles] for chunk in chunks)

Conociendo la variable objetivo y sus opciones

In [25]:
print(df['loan_status'].value_counts())

loan_status
Fully Paid                                             1076751
Current                                                 878317
Charged Off                                             268559
Late (31-120 days)                                       21467
In Grace Period                                           8436
Late (16-30 days)                                         4349
Does not meet the credit policy. Status:Fully Paid        1988
Does not meet the credit policy. Status:Charged Off        761
Default                                                     40
Name: count, dtype: int64


In [5]:
print(df['loan_status'].unique())

['Fully Paid' 'Current' 'Charged Off' 'In Grace Period'
 'Late (31-120 days)' 'Late (16-30 days)' 'Default' nan
 'Does not meet the credit policy. Status:Fully Paid'
 'Does not meet the credit policy. Status:Charged Off']


Preprocesamiento de la variable objetivo

In [26]:
#Convertimos el estado del préstamo a binario (1 = impago, 0 = pagado)
df['loan_status'] = df['loan_status'].apply(lambda x: 1 if x in ['Charged Off', 'Default', 'Late (31-120 days)', 'Late (16-30 days)'] else 0)


Preprocesamiento de las variables

In [27]:
#Variables numéricas y categóricas
vars_numericas = df.select_dtypes(include=['int64', 'float64']).drop('loan_status', axis=1).columns.tolist()
vars_categoricas = df.select_dtypes(include='object').columns.tolist()

#Imputación simple
df[vars_numericas] = df[vars_numericas].fillna(df[vars_numericas].median())
df[vars_categoricas] = df[vars_categoricas].fillna('Desconocido')

#Codificación
df = pd.get_dummies(df, columns=vars_categoricas, drop_first=True)

#Escalar en chunks
scaler = StandardScaler()
X_chunks = []
y_chunks = []

chunk_size = 100000  #Define un tamaño de chunk adecuado
for start in range(0, len(df), chunk_size):
	end = start + chunk_size
	chunk = df.iloc[start:end]
	X_chunk = scaler.fit_transform(chunk.drop('loan_status', axis=1))
	y_chunk = chunk['loan_status'].values
	X_chunks.append(X_chunk)
	y_chunks.append(y_chunk)

#Concatenar los resultados
X = np.vstack(X_chunks)
y = np.concatenate(y_chunks)

Dividir entre datos de entrenamiento y los de prueba

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

Creación de la red neuronal

In [29]:
model = Sequential()
model.add(Dense(64, activation='relu', input_dim=X.shape[1]))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

C:\Users\frank\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                 │ (None, 64)             │         6,528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 8,641 (33.75 KB)

 Trainable params: 8,641 (33.75 KB)

 Non-trainable params: 0 (0.00 B)

Entrenamiento de la red

In [30]:
history = model.fit(X_train, y_train, epochs=20, batch_size=64, validation_split=0.2)

Epoch 1/20
22607/22607 ━━━━━━━━━━━━━━━━━━━━ 36s 2ms/step - accuracy: 0.8904 - loss: 0.3052 - val_accuracy: 0.9320 - val_loss: 0.1981
Epoch 2/20
22607/22607 ━━━━━━━━━━━━━━━━━━━━ 34s 2ms/step - accuracy: 0.9266 - loss: 0.2156 - val_accuracy: 0.9350 - val_loss: 0.1890
Epoch 3/20
22607/22607 ━━━━━━━━━━━━━━━━━━━━ 33s 1ms/step - accuracy: 0.9295 - loss: 0.2125 - val_accuracy: 0.9361 - val_loss: 0.1864
Epoch 4/20
22607/22607 ━━━━━━━━━━━━━━━━━━━━ 34s 2ms/step - accuracy: 0.9312 - loss: 0.2049 - val_accuracy: 0.9372 - val_loss: 0.1835
Epoch 5/20
22607/22607 ━━━━━━━━━━━━━━━━━━━━ 34s 1ms/step - accuracy: 0.9324 - loss: 0.2027 - val_accuracy: 0.9385 - val_loss: 0.1813
Epoch 6/20
22607/22607 ━━━━━━━━━━━━━━━━━━━━ 32s 1ms/step - accuracy: 0.9329 - loss: 0.2012 - val_accuracy: 0.9392 - val_loss: 0.1794
Epoch 7/20
22607/22607 ━━━━━━━━━━━━━━━━━━━━ 36s 2ms/step - accuracy: 0.9337 - loss: 0.1993 - val_accuracy: 0.9402 - val_loss: 0.1778
Epoch 8/20
22607/22607 ━━━━━━━━━━━━━━━━━━━━ 37s 2ms/step - accuracy: 

Evaluación del modelo

In [31]:
y_pred = (model.predict(X_test) > 0.5).astype(int)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

14130/14130 ━━━━━━━━━━━━━━━━━━━━ 14s 951us/step
[[392675    583]
 [ 24476  34407]]
              precision    recall  f1-score   support

           0       0.94      1.00      0.97    393258
           1       0.98      0.58      0.73     58883

    accuracy                           0.94    452141
   macro avg       0.96      0.79      0.85    452141
weighted avg       0.95      0.94      0.94    452141



Probabilidad de impago

In [66]:
probabilidad_impago = model.predict(X_test)
print(probabilidad_impago[:10])  #Probabilidad para los primeros 10 casos
# Crear nombres genéricos para las columnas de X_test
column_names = [f'feature_{i}' for i in range(X_test.shape[1])]
df_test = pd.DataFrame(X_test, columns=column_names)
# Añadiendo las columnas de probabilidad y estado del préstamo al DataFrame
df_test['probabilidad_impago'] = probabilidad_impago.flatten()
df_test['loan_status'] = y_test
print(df_test.columns)


14130/14130 ━━━━━━━━━━━━━━━━━━━━ 11s 747us/step
[[0.04572156]
 [0.01494699]
 [0.04282269]
 [0.00214593]
 [0.7534716 ]
 [0.00517923]
 [0.04318774]
 [0.03640305]
 [0.98908305]
 [0.00963674]]
Index(['feature_0', 'feature_1', 'feature_2', 'feature_3', 'feature_4',
       'feature_5', 'feature_6', 'feature_7', 'feature_8', 'feature_9',
       ...
       'feature_93', 'feature_94', 'feature_95', 'feature_96', 'feature_97',
       'feature_98', 'feature_99', 'feature_100', 'probabilidad_impago',
       'loan_status'],
      dtype='object', length=103)


Respuestas a las preguntas del documento

Considere el rendimiento de una inversión correspondiente al retorno anual de la inversión.

In [54]:
#Paso 1: Calcular retorno bruto
df['retorno'] = (df['total_pymnt_inv'] / df['loan_amnt']) - 1

#Paso 2: Convertir 'term' a años
df['term_years'] = df['term'].astype(str).apply(lambda x: int(x.strip().split()[0])) / 12

#Paso 3: Calcular retorno anualizado
df['retorno_anual'] = ((df['retorno'] + 1) ** (1 / df['term_years'])) - 1

#Ver retorno anual promedio
print("Retorno anual promedio:", df['retorno_anual'].mean())


Retorno anual promedio: -0.08592878535578063


¿Los préstamos cancelados, generaron algún tipo de retorno?

In [43]:
# Filtrar los préstamos cancelados
cancelados = df[df['loan_status'] == 'Charged Off']

# Ver descripción estadística del retorno anual de préstamos cancelados
print(cancelados['retorno_anual'].describe())

count    268559.000000
mean         -0.183924
std           0.144814
min          -1.000000
25%          -0.263926
50%          -0.167006
75%          -0.079369
max           0.294712
Name: retorno_anual, dtype: float64


¿Cómo se explica la variación de los retornos de los créditos cancelados con respecto a la categoría del crédito?

In [44]:
# Agrupar préstamos cancelados por categoría y calcular retorno anual promedio
retorno_por_categoria = cancelados.groupby('grade')['retorno_anual'].mean()
print(retorno_por_categoria)

grade
A   -0.206839
B   -0.193214
C   -0.187275
D   -0.183619
E   -0.165887
F   -0.157511
G   -0.173076
Name: retorno_anual, dtype: float64


¿Existe diferencia entre el promedio de retorno de los créditos y la tasa de interés promedio? ¿Qué significa esta diferencia?

In [45]:
# Calcular promedios
retorno_promedio = df['retorno_anual'].mean()
tasa_promedio = df['int_rate'].mean()
diferencia = tasa_promedio - retorno_promedio

# Mostrar resultados
print("Tasa de interés promedio:", tasa_promedio)
print("Retorno anual promedio:", retorno_promedio)
print("Diferencia:", diferencia)

Tasa de interés promedio: 13.092829115111119
Retorno anual promedio: -0.08592878535578063
Diferencia: 13.1787579004669


¿Asumiendo el rol del inversionista, en cuáles créditos preferiría invertir y cuáles son las razones que motivan la decisión?

In [69]:
print(df.columns)
print(df_test.columns)

Index(['loan_amnt', 'term', 'int_rate', 'installment', 'grade', 'sub_grade',
       'emp_length', 'home_ownership', 'annual_inc', 'verification_status',
       'purpose', 'dti', 'delinq_2yrs', 'fico_range_low', 'fico_range_high',
       'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_record',
       'open_acc', 'total_acc', 'revol_bal', 'revol_util', 'application_type',
       'acc_now_delinq', 'pub_rec_bankruptcies', 'mort_acc',
       'num_tl_90g_dpd_24m', 'loan_status', 'total_pymnt_inv',
       'chargeoff_within_12_mths', 'retorno', 'term_years', 'retorno_anual'],
      dtype='object')
Index(['feature_0', 'feature_1', 'feature_2', 'feature_3', 'feature_4',
       'feature_5', 'feature_6', 'feature_7', 'feature_8', 'feature_9',
       ...
       'feature_93', 'feature_94', 'feature_95', 'feature_96', 'feature_97',
       'feature_98', 'feature_99', 'feature_100', 'probabilidad_impago',
       'loan_status'],
      dtype='object', length=103)


In [72]:
# Filtrar los créditos que el inversionista podría preferir


Construya al menos 3 atributos derivados que pueden utilizarse para predecir el impago del crédito. Justifique el atributo en términos del negocio. Justifique el nuevo atributo en términos del rendimiento del modelo.

In [77]:
#Atributo 1
df['ingreso_mensual'] = df['annual_inc'] / 12
#Atributo 2
df['cuota_ingreso']= df['installment'] / df['ingreso_mensual']
#Atributo 3
df['empleo_estable'] = df['emp_length'].apply(lambda x: 1 if str(x).strip().startswith(('5','6','7','8','9','10')) else 0)

#Ver resultado de ingreso mensual promedio
print("Ingreso mensual promedio:", df['ingreso_mensual'].mean())
#Ver resultado de cuota ingreso promedio
print("Cuota ingreso promedio:", df['cuota_ingreso'].mean())
#Ver resultado de empleo estable promedio
print("Empleo estable promedio:", df['empleo_estable'].mean())


Ingreso mensual promedio: 6499.369057255596
Cuota ingreso promedio: inf
Empleo estable promedio: 0.5548433870733016


¿Cuáles variables debe excluir del análisis porque se consideran fuga de datos?

¿Cuáles variables individuales son particularmente útiles para predecir la variable dependiente del estado del crédito?

Construya una medida que relacione cada una de las variables independientes con la variable predictora. Considere que la variable a predecir es binaria, Describa la interpretación de la medida.